In [1]:
import tensorflow as tf
import keras
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import time
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Flatten , Dense, Activation,Dropout
from keras.layers.advanced_activations import LeakyReLU

/home/vaishnavi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vaishnavi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vaishnavi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vaishnavi/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [3]:
print("Train samples: ",X_train.shape, y_train.shape)
print("Test samples: ",X_test.shape, y_test.shape)

Train samples:  (50000, 32, 32, 3) (50000, 1)
Test samples:  (10000, 32, 32, 3) (10000, 1)


In [4]:
labels = ["airplane", "automobile", "bird", "cat", "dog", "frog", "horse", "ship", "truck"]

In [5]:
X_train = (X_train/255) - 0.5
X_test = (X_test/255) - 0.5
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

In [6]:
def make_model(activation):
    model = Sequential()
    if activation == 'relu':
        model.add(Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
        model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
    if activation == 'tanh':
        model.add(Conv2D(16, (3,3), padding='same', activation='tanh', input_shape=(32,32,3)))
        model.add(Conv2D(32, (3,3), padding='same', activation='tanh', input_shape=(32,32,3)))
    if activation == 'sigmoid':
        model.add(Conv2D(16, (3,3), padding='same', activation='sigmoid', input_shape=(32,32,3)))
        model.add(Conv2D(32, (3,3), padding='same', activation='sigmoid', input_shape=(32,32,3)))
    model.add(MaxPooling2D(pool_size=(2,2), strides=None, padding='valid', data_format=None))
    model.add(Flatten())
    model.add(Dense(256, input_shape=(256, )))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    return model


In [7]:
# SGD without momentum
model = make_model('relu')
model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
             #optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
              metrics=['accuracy'])


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
_________________________________________________________________
activation_1 (Activation)    (None, 10)              

In [8]:
model.fit(X_train, y_train, epochs=10, batch_size=32,shuffle=False)


Epoch 1/10
50000/50000 [==============================] - 72s 1ms/step - loss: 1.7375 - accuracy: 0.3861
Epoch 2/10
50000/50000 [==============================] - 69s 1ms/step - loss: 1.3809 - accuracy: 0.5124
Epoch 3/10
50000/50000 [==============================] - 68s 1ms/step - loss: 1.2304 - accuracy: 0.5683
Epoch 4/10
50000/50000 [==============================] - 67s 1ms/step - loss: 1.1281 - accuracy: 0.6057
Epoch 5/10
50000/50000 [==============================] - 75s 2ms/step - loss: 1.0474 - accuracy: 0.6349
Epoch 6/10
50000/50000 [==============================] - 68s 1ms/step - loss: 0.9812 - accuracy: 0.6564
Epoch 7/10
50000/50000 [==============================] - 75s 1ms/step - loss: 0.9260 - accuracy: 0.6773
Epoch 8/10
50000/50000 [==============================] - 76s 2ms/step - loss: 0.8799 - accuracy: 0.6953
Epoch 9/10
50000/50000 [==============================] - 77s 2ms/step - loss: 0.8405 - accuracy: 0.7096
Epoch 10/10
50000/50000 [=============================

In [9]:
accuracy_sgd = model.evaluate(X_test, y_test, batch_size=32)
print("Test Loss: ",accuracy_sgd[0])
print("Test Accuracy: ",accuracy_sgd[1]*100)

10000/10000 [==============================] - 4s 352us/step
Test Loss:  1.0162196165084838
Test Accuracy:  65.27000069618225


In [10]:
# SGD with momentum
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False),
              metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, epochs=10, batch_size=32,shuffle=False)

Epoch 1/10
50000/50000 [==============================] - 66s 1ms/step - loss: 1.0562 - accuracy: 0.6324
Epoch 2/10
50000/50000 [==============================] - 67s 1ms/step - loss: 0.8982 - accuracy: 0.6882
Epoch 3/10
50000/50000 [==============================] - 66s 1ms/step - loss: 0.8057 - accuracy: 0.7219
Epoch 4/10
50000/50000 [==============================] - 67s 1ms/step - loss: 0.7331 - accuracy: 0.7472
Epoch 5/10
50000/50000 [==============================] - 67s 1ms/step - loss: 0.6657 - accuracy: 0.7718
Epoch 6/10
50000/50000 [==============================] - 67s 1ms/step - loss: 0.6081 - accuracy: 0.7871
Epoch 7/10
50000/50000 [==============================] - 67s 1ms/step - loss: 0.5693 - accuracy: 0.7992
Epoch 8/10
50000/50000 [==============================] - 67s 1ms/step - loss: 0.5403 - accuracy: 0.8074
Epoch 9/10
50000/50000 [==============================] - 67s 1ms/step - loss: 0.5151 - accuracy: 0.8182
Epoch 10/10
50000/50000 [==============================

In [12]:
accuracy_momentum = model.evaluate(X_test, y_test, batch_size=32)
print("Test Loss: ",accuracy_momentum[0])
print("Test Accuracy: ",accuracy_momentum[1]*100)

10000/10000 [==============================] - 4s 351us/step
Test Loss:  2.2852809394836426
Test Accuracy:  57.30999708175659


In [13]:
# SGD with Nesterov momentum
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, epochs=10, batch_size=32,shuffle=False)

Epoch 1/10
50000/50000 [==============================] - 83s 2ms/step - loss: 0.4009 - accuracy: 0.8568
Epoch 2/10
50000/50000 [==============================] - 83s 2ms/step - loss: 0.3730 - accuracy: 0.8680
Epoch 3/10
50000/50000 [==============================] - 83s 2ms/step - loss: 0.3646 - accuracy: 0.8725
Epoch 4/10
50000/50000 [==============================] - 83s 2ms/step - loss: 0.3575 - accuracy: 0.8785
Epoch 5/10
50000/50000 [==============================] - 83s 2ms/step - loss: 0.3361 - accuracy: 0.8854
Epoch 6/10
50000/50000 [==============================] - 83s 2ms/step - loss: 0.3368 - accuracy: 0.8857
Epoch 7/10
50000/50000 [==============================] - 83s 2ms/step - loss: 0.3168 - accuracy: 0.8946
Epoch 8/10
50000/50000 [==============================] - 83s 2ms/step - loss: 0.3218 - accuracy: 0.8950
Epoch 9/10
50000/50000 [==============================] - 83s 2ms/step - loss: 0.3107 - accuracy: 0.8991
Epoch 10/10
50000/50000 [==============================

In [15]:
accuracy_nesterov = model.evaluate(X_test, y_test, batch_size=32)
print("Test Loss: ",accuracy_nesterov[0])
print("Test Accuracy: ",accuracy_nesterov[1]*100)

10000/10000 [==============================] - 3s 350us/step
Test Loss:  3.9036438304901124
Test Accuracy:  57.30000138282776


In [16]:
def make_model_BN(activation):
    model = Sequential()
    if activation == 'relu':
        model.add(Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)))
        model.add(BatchNormalization())
    if activation == 'tanh':
        model.add(Conv2D(16, (3,3), padding='same', activation='tanh', input_shape=(32,32,3)))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3,3), padding='same', activation='tanh', input_shape=(32,32,3)))
        model.add(BatchNormalization())
    if activation == 'sigmoid':
        model.add(Conv2D(16, (3,3), padding='same', activation='sigmoid', input_shape=(32,32,3)))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3,3), padding='same', activation='sigmoid', input_shape=(32,32,3)))
        model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2), strides=None, padding='valid', data_format=None))
    model.add(Flatten())
    model.add(Dense(256, input_shape=(256, )))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    return model


In [17]:
model_BN = make_model_BN('relu')
model_BN.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 16)        448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 16)        64        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 32)        4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)              

In [18]:
# SGD with Batch Normalization without momentum
model_BN.compile(loss='categorical_crossentropy',
              optimizer='sgd',
             #optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [19]:
model_BN.fit(X_train, y_train, epochs=10, batch_size=32,shuffle=False)

Epoch 1/10
50000/50000 [==============================] - 112s 2ms/step - loss: 1.5736 - accuracy: 0.5084
Epoch 2/10
50000/50000 [==============================] - 112s 2ms/step - loss: 1.0421 - accuracy: 0.6375
Epoch 3/10
50000/50000 [==============================] - 132s 3ms/step - loss: 0.9226 - accuracy: 0.6803
Epoch 4/10
50000/50000 [==============================] - 137s 3ms/step - loss: 0.8460 - accuracy: 0.7081
Epoch 5/10
50000/50000 [==============================] - 127s 3ms/step - loss: 0.7855 - accuracy: 0.7298
Epoch 6/10
50000/50000 [==============================] - 112s 2ms/step - loss: 0.7322 - accuracy: 0.7482
Epoch 7/10
50000/50000 [==============================] - 112s 2ms/step - loss: 0.6841 - accuracy: 0.7662
Epoch 8/10
50000/50000 [==============================] - 112s 2ms/step - loss: 0.6396 - accuracy: 0.7825
Epoch 9/10
50000/50000 [==============================] - 112s 2ms/step - loss: 0.5971 - accuracy: 0.7992
Epoch 10/10
50000/50000 [=====================

In [20]:
accuracy_sgdBN = model_BN.evaluate(X_test, y_test, batch_size=32)
print("Test Loss: ",accuracy_sgdBN[0])
print("Test Accuracy: ",accuracy_sgdBN[1]*100)

10000/10000 [==============================] - 6s 627us/step
Test Loss:  1.2742890957832336
Test Accuracy:  63.440001010894775


In [21]:
# SGD with Batch Normalization + momentum
model_BN.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=False),
              metrics=['accuracy'])

In [22]:
model_BN.fit(X_train, y_train, epochs=10, batch_size=32,shuffle=False)

Epoch 1/10
50000/50000 [==============================] - 112s 2ms/step - loss: 1.0540 - accuracy: 0.6327
Epoch 2/10
50000/50000 [==============================] - 112s 2ms/step - loss: 0.8379 - accuracy: 0.7095
Epoch 3/10
50000/50000 [==============================] - 112s 2ms/step - loss: 0.7331 - accuracy: 0.7472
Epoch 4/10
50000/50000 [==============================] - 112s 2ms/step - loss: 0.6580 - accuracy: 0.7743
Epoch 5/10
50000/50000 [==============================] - 112s 2ms/step - loss: 0.6214 - accuracy: 0.7859
Epoch 6/10
50000/50000 [==============================] - 116s 2ms/step - loss: 0.5798 - accuracy: 0.8029
Epoch 7/10
50000/50000 [==============================] - 112s 2ms/step - loss: 0.5744 - accuracy: 0.8070
Epoch 8/10
50000/50000 [==============================] - 124s 2ms/step - loss: 0.5432 - accuracy: 0.8194
Epoch 9/10
50000/50000 [==============================] - 132s 3ms/step - loss: 0.5430 - accuracy: 0.8249
Epoch 10/10
50000/50000 [=====================

In [23]:
accuracy_momentumBN = model_BN.evaluate(X_test, y_test, batch_size=32)
print("Test Loss: ",accuracy_momentumBN[0])
print("Test Accuracy: ",accuracy_momentumBN[1]*100)

10000/10000 [==============================] - 6s 628us/step
Test Loss:  3.3469668128967287
Test Accuracy:  51.84999704360962


In [24]:
# SGD with Batch Normalization + Nesterov momentum
model_BN.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

In [25]:
model_BN.fit(X_train, y_train, epochs=10, batch_size=32,shuffle=False)

Epoch 1/10
50000/50000 [==============================] - 122s 2ms/step - loss: 0.4579 - accuracy: 0.8500
Epoch 2/10
50000/50000 [==============================] - 121s 2ms/step - loss: 0.4519 - accuracy: 0.8582
Epoch 3/10
50000/50000 [==============================] - 121s 2ms/step - loss: 0.4643 - accuracy: 0.8535
Epoch 4/10
50000/50000 [==============================] - 121s 2ms/step - loss: 0.4702 - accuracy: 0.8540
Epoch 5/10
50000/50000 [==============================] - 122s 2ms/step - loss: 0.4699 - accuracy: 0.8548
Epoch 6/10
50000/50000 [==============================] - 121s 2ms/step - loss: 0.4795 - accuracy: 0.8572
Epoch 7/10
50000/50000 [==============================] - 121s 2ms/step - loss: 0.4764 - accuracy: 0.8510
Epoch 8/10
50000/50000 [==============================] - 122s 2ms/step - loss: 0.4984 - accuracy: 0.8550
Epoch 9/10
50000/50000 [==============================] - 121s 2ms/step - loss: 0.5033 - accuracy: 0.8555
Epoch 10/10
50000/50000 [=====================

In [26]:
accuracy_nesterovBN = model_BN.evaluate(X_test, y_test, batch_size=32)
print("Test Loss: ",accuracy_nesterovBN[0])
print("Test Accuracy: ",accuracy_nesterovBN[1]*100)

10000/10000 [==============================] - 9s 873us/step
Test Loss:  3.056974808502197
Test Accuracy:  53.380000591278076
